In [2]:
# !pip install psychopg2

In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import time

keysku_errors=0
marketing_errors=0
online_errors=0
retail_errors=0
final_join_errors=0

In [4]:
key_sku_start_time = time.time()

datapath = "Resources/KEY_SKU.csv"
key_sku_df = pd.read_csv(datapath)
key_sku_df.head()

,Product SKU,StockCode
0,GGOENEBQ079099,21421
1,GGOENEBQ079199,21422
2,GGOENEBQ084699,22178
3,GGOEGAEB091117,20749
4,GGOENEBB078899,21056


In [5]:
# key_sku_df.dtypes

In [6]:
marketing_start_time = time.time()

datapath = "Resources/Marketing_Spend.csv"
marketing_df = pd.read_csv(datapath)
marketing_df.head()

,Unnamed: 0,Offline Spend,Online Spend
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


In [7]:
online_start_time = time.time()

datapath = "Resources/Online.csv"
online_df = pd.read_csv(datapath)
online_df.head(3)

,Transaction ID,Date,Product SKU,Product,Product Category (Enhanced E-commerce),Quantity,Avg. Price,Revenue,Tax,Delivery
0,48497,20171231,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.0,34.44,19.99
1,48496,20171231,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.0,33.14,6.50
2,48495,20171231,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.0,12.06,6.50


In [8]:
# online_df.dtypes

In [9]:
retail_start_time = time.time()

datapath = "Resources/Retail.csv"
retail_df = pd.read_csv(datapath)
retail_df.head(3)

,InvoiceNo,InvoiceDate,StockCode,Quantity
0,536598,2017-01-01,21421,1
1,536598,2017-01-01,21422,2
2,536598,2017-01-01,22178,26


In [10]:
# Create a filtered dataframe from specific columns
try:
    key_sku_transformed= key_sku_df.rename(columns={"Product SKU":"product_sku",
                                                "StockCode":"stock_code"})    
except:
    keysku_errors+=1
    
key_sku_transformed.dropna()
key_sku_transformed.head(3)

,product_sku,stock_code
0,GGOENEBQ079099,21421
1,GGOENEBQ079199,21422
2,GGOENEBQ084699,22178


In [11]:
# key_sku_transformed.dtypes

In [12]:
#create report count
key_sku_count=key_sku_transformed.shape[0]


In [13]:
retail_df['InvoiceDate']=pd.to_datetime(retail_df['InvoiceDate'].astype(str), format='%Y/%m/%d')


In [14]:
# Create a filtered dataframe from specific columns
try:
    retail_transformed= retail_df.rename(columns={"InvoiceNo": "invoice_no", 
                                                    "InvoiceDate":"invoice_date", 
                                                    "StockCode":"stock_code", 
                                                  "Quantity":"quantity"})
    
except:
    retail_errors+=1

retail_transformed.dropna()
retail_transformed.head(3)

,invoice_no,invoice_date,stock_code,quantity
0,536598,2017-01-01,21421,1
1,536598,2017-01-01,21422,2
2,536598,2017-01-01,22178,26


In [15]:
#create report count
retail_count=retail_transformed.shape[0]


In [16]:
# retail_transformed.dtypes

In [17]:
marketing_df['Unnamed: 0']=pd.to_datetime(marketing_df['Unnamed: 0'].astype(str), format='%Y/%m/%d')


In [18]:
# marketing_df.dtypes

In [19]:
# Create a filtered dataframe from specific columns
try:
    marketing_transformed= marketing_df.rename(columns={"Unnamed: 0": "invoice_date", 
                                                                 "Offline Spend":"offline_spending", 
                                                                 "Online Spend":"online_spending"})
    
except:
    marketing_errors+=1

marketing_transformed.dropna()
marketing_transformed.head(3)

,invoice_date,offline_spending,online_spending
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38


In [20]:
# marketing_transformed.dtypes

In [21]:
# format spending columns as $xx.xx
# marketing_transformed['online_spending'] = marketing_transformed['online_spending'].astype(int)
# marketing_transformed['offline_spending'] = marketing_transformed['offline_spending'].astype(int)
# marketing_df['date']=pd.to_datetime(marketing_df['date'].astype(str), format='%Y/%m/%d')
# marketing_transformed.head()


In [22]:
#create report count
marketing_count=marketing_transformed.shape[0]


In [23]:
# online_df.describe()

In [24]:
online_df['Date']=pd.to_datetime(online_df['Date'].astype(str), format='%Y/%m/%d')

# Create a filtered dataframe from specific columns
try:
    
# Rename the column headers
    online_transformed=online_df.rename(columns={"Transaction ID":"transaction_id", 
                                                 "Date":"invoice_date", 
                                                 "Product SKU":"product_sku",
                                                 "Product":"product",
                                                 "Product Category (Enhanced E-commerce)":"product_category",
                                                 "Quantity":"quantity",
                                                 "Avg. Price":"ave_price",
                                                 "Revenue":"revenue",
                                                 "Tax":"tax",
                                                 "Delivery":"delivery_cost"})
     

except:
    online_errors+=1

online_transformed.dropna()
online_transformed.head(3)


,transaction_id,invoice_date,product_sku,product,product_category,quantity,ave_price,revenue,tax,delivery_cost
0,48497,2017-12-31,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.0,34.44,19.99
1,48496,2017-12-31,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.0,33.14,6.50
2,48495,2017-12-31,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.0,12.06,6.50


In [25]:
# online_transformed.dtypes

In [26]:
# create report count
online_count=online_transformed.shape[0]


In [27]:
# inner join key_sku to retail on stock_code

# then inner join new retail_db on to online_db on product sku

# then on marketing join on invoice date after changing the name

In [28]:
# Join key_sku and retail
sku_retail_join=pd.merge(key_sku_transformed, retail_transformed, 
                        on='stock_code', how='inner')

# Set index
sku_retail_join.set_index('invoice_no', inplace=True)
sku_retail_join.head(3)

,product_sku,stock_code,invoice_date,quantity
invoice_no,,,,
536598,GGOENEBQ079099,21421,2017-01-01,1
536999,GGOENEBQ079099,21421,2017-01-02,1
537137,GGOENEBQ079099,21421,2017-01-04,2


In [29]:
# sku_retail_join.dtypes

In [30]:
# Merge key_sku and online
sku_online_join=pd.merge(key_sku_transformed, online_transformed, 
                        on='product_sku', how='inner')

# Set index
sku_online_join.set_index('product_sku', inplace=True)
sku_online_join.head(3)

,stock_code,transaction_id,invoice_date,product,product_category,quantity,ave_price,revenue,tax,delivery_cost
product_sku,,,,,,,,,,
GGOENEBQ079099,21421,48497,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.0,34.44,19.99
GGOENEBQ079099,21421,48491,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,2.0,80.52,376.0,0.00,6.50
GGOENEBQ079099,21421,48475,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,1022.0,92.58,6.50


In [31]:
# Merge new sku_stock_join and online
retail_online_join=pd.merge(sku_retail_join, online_transformed, 
                        on='invoice_date', how='inner')

retail_online_join.head(3)

,product_sku_x,stock_code,invoice_date,quantity_x,transaction_id,product_sku_y,product,product_category,quantity_y,ave_price,revenue,tax,delivery_cost
0,GGOENEBQ079099,21421,2017-01-01,1,16736,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,10.89,6.5
1,GGOENEBQ079099,21421,2017-01-01,1,16733,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5
2,GGOENEBQ079099,21421,2017-01-01,1,16732,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,13.22,6.5


In [32]:
# retail_online_join.dtypes

In [33]:
# Rename and drop unnecessary columns
new_retail_online=retail_online_join.rename(columns={"quantity_x":"quantity_retail",
                                 "quantity_y":"quantity_online",
                                 "product_sku_x":"product_sku"})\
                                .drop(columns=['product_sku_y'])
new_retail_online.head(2)


,product_sku,stock_code,invoice_date,quantity_retail,transaction_id,product,product_category,quantity_online,ave_price,revenue,tax,delivery_cost
0,GGOENEBQ079099,21421,2017-01-01,1,16736,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,10.89,6.5
1,GGOENEBQ079099,21421,2017-01-01,1,16733,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5


In [34]:
# Marketing join on new_retail
final_join=pd.merge(new_retail_online, marketing_transformed, 
                        on='invoice_date', how='inner')

final_join.head(2)

,product_sku,stock_code,invoice_date,quantity_retail,transaction_id,product,product_category,quantity_online,ave_price,revenue,tax,delivery_cost,offline_spending,online_spending
0,GGOENEBQ079099,21421,2017-01-01,1,16736,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,10.89,6.5,4500,2424.5
1,GGOENEBQ079099,21421,2017-01-01,1,16733,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5,4500,2424.5


In [35]:
# final_join.dtypes

In [36]:
# Set index
final_join.set_index('invoice_date', inplace=True)
final_join.head(3)

,product_sku,stock_code,quantity_retail,transaction_id,product,product_category,quantity_online,ave_price,revenue,tax,delivery_cost,offline_spending,online_spending
invoice_date,,,,,,,,,,,,,
2017-01-01,GGOENEBQ079099,21421,1,16736,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,10.89,6.5,4500,2424.5
2017-01-01,GGOENEBQ079099,21421,1,16733,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5,4500,2424.5
2017-01-01,GGOENEBQ079099,21421,1,16732,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,13.22,6.5,4500,2424.5


In [38]:
# Connect to database
import psycopg2 as psycopg

In [39]:
# conn = psycopg.connect("dbname=ETL-challenge user=postgres password=postgres")
connection_string="postgres:postgres@localhost:5432/ETL-challenge"

In [40]:
# engine = create_engine(f'postgresql://{conn}')
engine = create_engine(f'postgresql://{connection_string}')

In [41]:
engine.table_names()

['key_sku', 'retail', 'marketing', 'online']

In [42]:
# Load dataframes into database

key_sku_transformed.to_sql(name='key_sku', con=engine, if_exists='append', index=True)
key_sku_end_time=time.time() - key_sku_start_time

In [43]:

retail_transformed.to_sql(name='retail', con=engine, if_exists='append', index=True)
retail_end_time=time.time() - retail_start_time

In [44]:

marketing_transformed.to_sql(name='marketing', con=engine, if_exists='append', index=True)
marketing_end_time=time.time() - marketing_start_time

In [45]:

online_transformed.to_sql(name='online', con=engine, if_exists='append', index=True)
online_end_time=time.time() - online_start_time

In [46]:
print(key_sku_end_time)
print(retail_end_time)
print(marketing_end_time)
print(online_end_time)

16.738070964813232
34.64468002319336
34.81864857673645
41.898030042648315


In [47]:
print(key_sku_count)
print(retail_count)
print(marketing_count)
print(online_count)
print(final_join_count)

1178
181247
365
54144


In [48]:
print(keysku_errors)
print(retail_errors)
print(marketing_errors)
print(online_errors)

0
0
0
0


In [49]:
# Confirm data has been added by querying a table
pd.read_sql_query('SELECT * FROM online', con=engine).head(3)

,index,transaction_id,invoice_date,product_sku,product,product_category,quantity,ave_price,revenue,tax,delivery_cost
0,0,48497,2017-12-31,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.0,34.44,19.99
1,1,48496,2017-12-31,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.0,33.14,6.50
2,2,48495,2017-12-31,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.0,12.06,6.50


In [50]:
danyelle_etl_report=pd.DataFrame({'domain': ['key_sku','retail','marketing','online'],
                        'row_counts': [key_sku_count, retail_count, 
                                       marketing_count, online_count],
                        'processing_time':[str(round(key_sku_end_time,2)) + ' seconds',
                                          str(round(retail_end_time,2)) + ' seconds',
                                          str(round(marketing_end_time,2)) + ' seconds',
                                          str(round(online_end_time,2)) + ' seconds'],
                        'error_count':[keysku_errors, retail_errors, 
                                       marketing_errors, online_errors]})

In [51]:
danyelle_etl_report.head()

,domain,row_counts,processing_time,error_count
0,key_sku,1178,16.74 seconds,0
1,retail,181247,34.64 seconds,0
2,marketing,365,34.82 seconds,0
3,online,54144,41.9 seconds,0


In [52]:
danyelle_etl_report.to_html('danyelle_etl_report.html', index=False)